# CUDA-Accelerated RCPSP Solver using Genetic Algorithm

This notebook implements a GPU-accelerated solver for the Resource-Constrained Project Scheduling Problem (RCPSP) using CUDA and genetic algorithms. The solver is designed to handle multiple instances efficiently by leveraging parallel processing capabilities of modern GPUs.

## Key Features:
- **CUDA Acceleration**: Utilizes CuPy for GPU-accelerated computations
- **Genetic Algorithm**: Evolutionary approach for finding optimal solutions
- **Batch Processing**: Handles multiple RCPSP instances simultaneously
- **Performance Optimization**: Optimized memory access patterns and kernel designs

In [ ]:
!pip install cupy-cuda12x

In [ ]:
import cupy as cp
import numpy as np
import pandas as pd
import time
import os
import glob
from pathlib import Path
import re
from typing import List, Tuple, Dict, Optional
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

In [ ]:
@dataclass
class RCPSPInstance:
    """Data structure for RCPSP instance"""
    n_jobs: int
    n_resources: int
    durations: np.ndarray
    resource_demands: np.ndarray
    resource_capacities: np.ndarray
    precedence_matrix: np.ndarray
    successors: List[List[int]]
    predecessors: List[List[int]]

def parse_rcpsp_file(filepath: str) -> RCPSPInstance:
    """Parse RCPSP .data file and return instance structure"""
    with open(filepath, 'r') as f:
        lines = f.readlines()

    # Find key sections
    jobs_line = None
    resources_line = None
    precedence_start = None
    requests_start = None
    capacities_start = None

    for i, line in enumerate(lines):
        if 'jobs (incl. supersource/sink )' in line:
            jobs_line = i
        elif 'renewable resource types' in line:
            resources_line = i
        elif 'PRECEDENCE RELATIONS' in line:
            precedence_start = i
        elif 'REQUESTS/DURATIONS' in line:
            requests_start = i
        elif 'RESOURCEAVAILABILITIES' in line:
            capacities_start = i

    # Extract basic info
    n_jobs = int(lines[jobs_line].split()[0])
    n_resources = int(lines[resources_line].split()[0])

    # Parse durations and resource demands
    durations = np.zeros(n_jobs)
    resource_demands = np.zeros((n_jobs, n_resources))

    for i in range(requests_start + 3, requests_start + 3 + n_jobs):
        parts = lines[i].split()
        job_id = int(parts[0]) - 1
        durations[job_id] = int(parts[2])
        for r in range(n_resources):
            resource_demands[job_id, r] = int(parts[3 + r])

    # Parse resource capacities
    capacity_line = lines[capacities_start + 2].split()
    resource_capacities = np.array([int(x) for x in capacity_line])

    # Parse precedence relations
    successors = [[] for _ in range(n_jobs)]
    predecessors = [[] for _ in range(n_jobs)]

    for i in range(precedence_start + 3, precedence_start + 3 + n_jobs):
        parts = lines[i].split()
        job_id = int(parts[0]) - 1
        n_successors = int(parts[2])
        for j in range(n_successors):
            succ_id = int(parts[3 + j]) - 1
            successors[job_id].append(succ_id)
            predecessors[succ_id].append(job_id)

    # Build precedence matrix
    precedence_matrix = np.zeros((n_jobs, n_jobs), dtype=bool)
    for i in range(n_jobs):
        for j in successors[i]:
            precedence_matrix[i, j] = True

    return RCPSPInstance(
        n_jobs=n_jobs,
        n_resources=n_resources,
        durations=durations,
        resource_demands=resource_demands,
        resource_capacities=resource_capacities,
        precedence_matrix=precedence_matrix,
        successors=successors,
        predecessors=predecessors
    )

def load_multiple_instances(data_dir: str) -> Dict[str, RCPSPInstance]:
    """Load multiple RCPSP instances from directory"""
    instances = {}
    data_files = glob.glob(os.path.join(data_dir, "*.data"))

    for filepath in data_files:
        instance_name = Path(filepath).stem
        try:
            instances[instance_name] = parse_rcpsp_file(filepath)
            print(f"Loaded {instance_name}: {instances[instance_name].n_jobs} jobs, {instances[instance_name].n_resources} resources")
        except Exception as e:
            print(f"Error loading {instance_name}: {e}")



In [ ]:
# CUDA kernels for fitness evaluation
fitness_kernel_code = '''
extern "C" __global__
void evaluate_fitness(float* schedules, float* durations, float* resource_demands,
                     float* resource_capacities, bool* precedence_matrix,
                     float* fitness_values, int pop_size, int n_jobs, int n_resources) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= pop_size) return;

    float* schedule = schedules + idx * n_jobs;
    float makespan = 0.0f;
    bool feasible = true;

    // Check precedence constraints
    for (int i = 0; i < n_jobs && feasible; i++) {
        for (int j = 0; j < n_jobs; j++) {
            if (precedence_matrix[i * n_jobs + j] && schedule[i] >= schedule[j]) {
                feasible = false;
                break;
            }
        }
    }

    if (feasible) {
        // Calculate makespan and check resource constraints
        for (int i = 0; i < n_jobs; i++) {
            float finish_time = schedule[i] + durations[i];
            if (finish_time > makespan) {
                makespan = finish_time;
            }
        }

        // Check resource constraints at each time point
        int max_time = (int)(makespan + 1);
        for (int t = 0; t < max_time && feasible; t++) {
            for (int r = 0; r < n_resources; r++) {
                float resource_usage = 0.0f;
                for (int j = 0; j < n_jobs; j++) {
                    if (schedule[j] <= t && t < schedule[j] + durations[j]) {
                        resource_usage += resource_demands[j * n_resources + r];
                    }
                }
                if (resource_usage > resource_capacities[r]) {
                    feasible = false;
                    break;
                }
            }
        }
    }

    fitness_values[idx] = feasible ? makespan : 1e6f + makespan;
}
'''

crossover_kernel_code = '''
extern "C" __global__
void crossover_mutation(float* population, float* new_population, curandState* states,
                       float* durations, bool* precedence_matrix,
                       int pop_size, int n_jobs, float crossover_rate, float mutation_rate) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= pop_size) return;

    curandState localState = states[idx];
    float* individual = population + idx * n_jobs;
    float* new_individual = new_population + idx * n_jobs;

    // Copy original individual
    for (int i = 0; i < n_jobs; i++) {
        new_individual[i] = individual[i];
    }

    // Crossover
    if (curand_uniform(&localState) < crossover_rate) {
        int partner